In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
# Set random seed for reproducibility
torch.manual_seed(31)
np.random.seed(31)

In [59]:
# Load the CSV file
df = pd.read_csv('north_korea_missile_test_database.csv')

df = df.drop(columns = ["F1","Date", "Date Entered/Updated", "Other Name", "Additional Information", "Source(s)"], axis = 1, errors = "ignore")


In [60]:
# Clean data
df = df.replace(['Unknown', ''], np.nan)

# Clean numerical columns
for col in ['Apogee', 'Distance Travelled','Facility Latitude', 'Facility Longitude']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(' km', '', regex=False).str.replace(',', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Handle missing values
numerical_cols = ['Facility Latitude', 'Facility Longitude', 'Apogee', 'Distance Travelled']
categorical_cols = ["Launch Time (UTC)",'Missile Type', 'Launch Agency/Authority', 
                   'Facility Name', 'Facility Location', 'Landing Location', 
                   'Confirmation Status', 'Test Outcome']

for col in numerical_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')
print(df.dtypes)
df.tail(10)


Launch Time (UTC)           object
Missile Name                object
Missile Type                object
Launch Agency/Authority     object
Facility Name               object
Facility Location           object
Facility Latitude          float64
Facility Longitude         float64
Landing Location            object
Apogee                     float64
Distance Travelled         float64
Confirmation Status         object
Test Outcome                object
dtype: object


,Launch Time (UTC),Missile Name,Missile Type,Launch Agency/Authority,Facility Name,Facility Location,Facility Latitude,Facility Longitude,Landing Location,Apogee,Distance Travelled,Confirmation Status,Test Outcome
125,08:34:00 PM,KN-24,SRBM,Unknown,Hungnam,"Hamhung, South Hamgyong",39.811611,127.663750,Sea of Japan or East Sea,48.000000,400.000000,Confirmed,Success
126,08:50:00 PM,KN-24,SRBM,Unknown,Hungnam,"Hamhung, South Hamgyong",39.811611,127.663750,Sea of Japan or East Sea,48.000000,400.000000,Confirmed,Success
127,11:01:00 PM,KN-24,SRBM,Unknown,Tongchan,"Kangwon Province, (North Korea)",38.953797,127.891882,Sea of Japan or East Sea,30.000000,230.000000,Confirmed,Success
128,11:16:00 PM,KN-24,SRBM,Unknown,Tongchan,"Kangwon Province, (North Korea)",38.953797,127.891882,Sea of Japan or East Sea,30.000000,230.000000,Confirmed,Success
129,09:45:00 PM,KN-25,SRBM,Unknown,Sondok Airbase,"Sondok, South Hamgyong Province",39.743700,127.473200,Sea of Japan or East Sea,97.000000,380.000000,Confirmed,Success
130,10:02:00 PM,KN-25,SRBM,Unknown,Sondok Airbase,"Sondok, South Hamgyong Province",39.743700,127.473200,Sea of Japan or East Sea,97.000000,380.000000,Confirmed,Success
131,09:53:00 PM,KN-25,SRBM,Unknown,Kaechon Air Base,"Kaechon, South Pyongan Province",39.752321,125.899905,Sea of Japan or East Sea,50.000000,330.000000,Confirmed,Success
132,10:12:00 PM,KN-25,SRBM,Unknown,Kaechon Air Base,"Kaechon, South Pyongan Province",39.752321,125.899905,Sea of Japan or East Sea,50.000000,330.000000,Confirmed,Success
133,Unknown,KN-25,SRBM,Unknown,Kaechon Air Base,"Kaechon, South Pyongan Province",39.752321,125.899905,Sea of Japan or East Sea,384.627273,629.103448,Confirmed,Failure
134,10:00:00 PM,Pukguksong-3,SLBM,Unknown,Yonghung Bay,"Kangwon Province, (North Korea)",39.260000,127.580000,Sea of Japan or East Sea,910.000000,450.000000,Confirmed,Success


In [61]:
# Target
target = 'Missile Name'
y = df[target]
X = df.drop(columns=[target])


In [62]:
# Encode categoricals with LabelEncoder
label_encoders = {}
for col in categorical_cols:
    if col in X.columns:
        label = LabelEncoder()
        X[col] = label.fit_transform(X[col].astype(str))
        label_encoders[col] = label 
# Scale numerical features
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Encode target
label_target = LabelEncoder()
y_encoded = label_target.fit_transform(y)
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)


In [63]:
# Split data: 60% train, 20% val, 20% test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=31)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=31)

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val.values)
y_val_tensor = torch.LongTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.LongTensor(y_test)


In [64]:
# Datasets
batch_size = 32  # Adjust based on dataset size
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
val_ds = TensorDataset(X_val_tensor, y_val_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)

# Dataloaders
train_loader = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_ds, batch_size = batch_size)
test_loader = DataLoader(test_ds, batch_size = batch_size)


In [65]:
# Define the neural network
class MissileClassifier(nn.Module):
    def __init__(self, input_size, num_classes, h1 = 64, h2 = 32):
        super().__init__()
        self.fc1 = nn.Linear(input_size, h1)
        self.bn1 = nn.BatchNorm1d(h1) # Adding batch normalization
        self.dropout1 = nn.Dropout(0.5)  # Adding a dropout layer to regulate overfitting
        self.fc2 = nn.Linear(h1, h2)
        self.bn2 = nn.BatchNorm1d(h2)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(h2, num_classes)
       
          
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        return self.fc3(x)

input_size = X.shape[1]
num_classes = len(label_target.classes_)
model = MissileClassifier(input_size, num_classes)

In [66]:
# Loss with class weights, optimizer with weight decay, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  


In [67]:
# Training loop
epochs = 100  
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
         # Keep track of our losses
        losses.append(loss.detach().numpy())
        loss.backward()
        optimizer.step()

       
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
    val_loss /= len(val_loader)
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}/{epochs}, Val Loss: {val_loss:.4f}')


Epoch 0/100, Val Loss: 3.2936
Epoch 10/100, Val Loss: 2.7092
Epoch 20/100, Val Loss: 2.4883
Epoch 30/100, Val Loss: 2.3751
Epoch 40/100, Val Loss: 2.3392
Epoch 50/100, Val Loss: 2.3050
Epoch 60/100, Val Loss: 2.2522
Epoch 70/100, Val Loss: 2.1836
Epoch 80/100, Val Loss: 2.1265
Epoch 90/100, Val Loss: 2.0773


In [68]:
# Test the model
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())
        y_true.extend(labels.numpy())

accuracy = accuracy_score(y_true, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 74.07%
